# Предсказание способа обезвреживания преступника

In [37]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split 

In [38]:
df = pd.read_csv('police_use_of_force.csv')

In [39]:
df.head()

,X,Y,PoliceUseOfForceID,CaseNumber,ResponseDate,Problem,Is911Call,PrimaryOffense,SubjectInjury,ForceReportNumber,...,TotalCityCallsForYear,TotalPrecinctCallsForYear,TotalNeighborhoodCallsForYear,CenterGBSID,CenterLatitude,CenterLongitude,CenterX,CenterY,DateAdded,OBJECTID
0,-93.293351,44.999153,11675689,08-005035,2008/01/05 17:08:18+00,Traffic Law Enforcement,No,FLEE,No,3,...,322402,80434.0,10077,16119,44.999153,-93.293351,-1.038537e+07,5.621388e+06,2020/06/17 08:19:17+00,1
1,-93.251092,44.961813,11675690,08-012774,2008/01/13 03:21:52+00,Suspicious Vehicle,No,FLEE,No,1,...,322402,84018.0,10316,17023,44.961813,-93.251092,-1.038066e+07,5.615512e+06,2020/06/17 08:19:17+00,2
2,-93.266112,44.974295,11675691,08-019237,2008/01/20 03:47:57+00,Unwanted Person,Yes,OBSTRU,No,1,...,322402,46998.0,23458,21739,44.974295,-93.266112,-1.038234e+07,5.617476e+06,2020/06/17 08:19:17+00,3
3,-93.287968,44.962973,11675692,08-032801,2008/02/03 00:52:49+00,Narcotics,No,NARC,No,1,...,322402,55689.0,5110,16044,44.962973,-93.287968,-1.038477e+07,5.615694e+06,2020/06/17 08:19:17+00,4
4,-93.272499,44.979626,11675693,08-038886,2008/02/09 02:11:03+00,Fight,No,DISCON,NaN,1,...,322402,46998.0,23458,25832,44.979626,-93.272499,-1.038305e+07,5.618315e+06,2020/06/17 08:19:17+00,5


Посмотрим на информацию об этом датасете

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30129 entries, 0 to 30128
Data columns (total 30 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   X                              30129 non-null  float64
 1   Y                              30129 non-null  float64
 2   PoliceUseOfForceID             30129 non-null  int64  
 3   CaseNumber                     30129 non-null  object 
 4   ResponseDate                   30129 non-null  object 
 5   Problem                        28898 non-null  object 
 6   Is911Call                      28994 non-null  object 
 7   PrimaryOffense                 28996 non-null  object 
 8   SubjectInjury                  19422 non-null  object 
 9   ForceReportNumber              30129 non-null  int64  
 10  SubjectRole                    28980 non-null  object 
 11  SubjectRoleNumber              30111 non-null  float64
 12  ForceType                      28909 non-null 

В датасете есть пропущенные значения, поэтому удалим строки, где есть хотя юы одно пустое значение.

In [41]:
df.dropna(inplace=True)

Также в датасете много "лишних" колонок, которые будут мешать при предсказании. Удалим и их.

In [43]:
df.drop(columns=df.columns[range(0,5)] ,axis=1, inplace=True)

In [45]:
df.drop(columns=df.columns[range(-10, 0)], axis=1, inplace=True)

Посмотрим на данные еще раз.

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19171 entries, 0 to 28978
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Problem            19171 non-null  object 
 1   Is911Call          19171 non-null  object 
 2   PrimaryOffense     19171 non-null  object 
 3   SubjectInjury      19171 non-null  object 
 4   ForceReportNumber  19171 non-null  int64  
 5   SubjectRole        19171 non-null  object 
 6   SubjectRoleNumber  19171 non-null  float64
 7   ForceType          19171 non-null  object 
 8   ForceTypeAction    19171 non-null  object 
 9   Race               19171 non-null  object 
 10  Sex                19171 non-null  object 
 11  EventAge           19171 non-null  float64
 12  TypeOfResistance   19171 non-null  object 
 13  Precinct           19171 non-null  object 
 14  Neighborhood       19171 non-null  object 
dtypes: float64(2), int64(1), object(12)
memory usage: 2.3+ MB


Видно, что бОльшая часть данных типа object, но sklearn работает только с числовыми. Для работы модели подключим LabelEncoder и все колонки типа object перекодируем в int64.

In [78]:
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()

In [79]:
for column in df:
    if df[column].dtype == 'object':
        le.fit (df[column])
        df[column] = le.transform(df[column])

In [80]:
df.head()

,Problem,Is911Call,PrimaryOffense,SubjectInjury,ForceReportNumber,SubjectRole,SubjectRoleNumber,ForceType,ForceTypeAction,Race,Sex,EventAge,TypeOfResistance,Precinct,Neighborhood
0,104,0,93,0,3,0,1.0,1,18,1,1,23.0,6,4,56
1,98,0,93,0,1,0,1.0,10,13,1,1,30.0,6,3,76
2,111,1,145,0,1,0,1.0,2,32,1,1,40.0,4,1,19
3,60,0,138,0,1,0,1.0,1,18,1,1,30.0,8,5,72
6,34,1,145,0,1,0,1.0,1,40,1,1,17.0,15,4,31


In [81]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score

Разделим датасет на несколько групп для обучения и для теста.

In [73]:
X = df.drop('ForceType', axis=1)
y = df.ForceType

In [74]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,train_size=0.75 , random_state = 42)

Далее создадим дерево, проведем классификацию и проверим точность.

In [75]:
clas = DecisionTreeClassifier(criterion='entropy', max_depth=5) #инициация дерева
clas.fit(Xtrain, ytrain) #тренировка модели по тренировочным иксам
ypred = clas.predict(Xtest) #предсказания модели тестовых иксах
accuracy_score(ytest, ypred) #смотрим, насколько предсказанные У соотносятся с настоящими У-тест

0.9866471938243272